#Chapter2: Working with Text  


In [ ]:
from importlib.metadata import version
print("torch version:", version("torch"))
print("tiktoken version: ", version("tiktoken"))

torch version: 2.8.0+cu126
tiktoken version:  0.12.0


In [ ]:
# The veredict
import os
import urllib.request
url = ("https://raw.githubusercontent.com/rasbt/"
       "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
       "the-verdict.txt")
file_path = "the-veredict.txt"
urllib.request.urlretrieve(url, file_path)

('the-veredict.txt', <http.client.HTTPMessage at 0x7b17c5fd9670>)

In [ ]:
# Lectura breve del archivo usando python
with open(file_path, 'r', encoding='utf-8') as f:
    raw_text = f.read()

print("Numero total de caracteres: ", len(raw_text))
print("Primeros 100 caracteres: ", raw_text[:100])

Numero total de caracteres:  20479
Primeros 100 caracteres:  I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no g


In [ ]:
import re
text = "Hello, world. This is a test."
result = re.split(r'(\s)', text) # separar por espacios
print(result)
result_2 = re.split(r'([,.]|\s)', text)  # separar por espacios y punto y comas
print(result_2)

['Hello,', ' ', 'world.', ' ', 'This', ' ', 'is', ' ', 'a', ' ', 'test.']
['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [ ]:
# eliminar carater redundantes de maera segura como espacios en blanco
result_3 = [item for item in result_2 if item.strip()]
print(result_3)

['Hello', ',', 'world', '.', 'This', 'is', 'a', 'test', '.']


In [ ]:
# para eleminar signos de pregunas, admiracion dobles espacios
text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [ ]:
# Como trabajo un tokenizador basico
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()] # crea una tokenizadro a partir del texto
print(len(preprocessed))
print(preprocessed[:30])

4690
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [ ]:
# Convertir token en ID´s (Vocabulary)
all_words = sorted(set(preprocessed))   # organiza la lista de la variable preprocessed
vocab__size = len(all_words)
print(vocab__size)

1130


In [ ]:
# Creado  el vocabulario
vocab = {token: integer for integer, token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
  print(item)
  if i>=50:
    break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


In [ ]:
# Convertir numeros (ID) a texto mediante un tokenizador
# Tokenizador simple
class SimpleTokenizerV1:
  def __init__(self, vocab):
     self.str_to_int = vocab
     self.int_to_str = {i:s for s,i in vocab.items()}   # Crea un vacbulario inverso del map de los IDs

  def encode(self,text):      # Procesa el texto de entrada y lo convierte en identificadores de tokens.

    preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',text)
    preprocessed = [item.strip() for item in preprocessed if item.strip()]
    ids = [self.str_to_int[s] for s in preprocessed]
    return ids

  def decode(self, ids):     # Convierte tokenIDs  en texto
    text = " ".join([self.int_to_str[i] for i in ids])
    text = re.sub(r'([,.:;?_!"()\']|--|\s)', r'\1', text) # Remueve los espacios despues de carateres especificos
    return text


In [ ]:
tokenizador = SimpleTokenizerV1(vocab)
text = """"It's the last he painted, you know,"
       Mrs. Gisburn said with pardonable pride.
"""
ids = tokenizador.encode(text)
print('---------------------------------')
print(ids)
print('---------------------------------')
print(tokenizador.decode(ids))

---------------------------------
[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]
---------------------------------
" It ' s the last he painted , you know , " Mrs . Gisburn said with pardonable pride .


In [ ]:
new_text = "Hello, do you like tea?"
print(tokenizador.encode(new_text))

KeyError: 'Hello'

El problema es que la palabra “Hello” no se utilizó en el cuento corto “The Verdict”. Por lo tanto, no está incluida en el vocabulario.

In [ ]:
# Mejorando el tokenizador para menajar palabras desconocidas que no
# hicieron parte del vocab <unknow> y para separar fuentes de texto diferentes
# <|endoftext|>

all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token: integer for integer, token in enumerate(all_tokens)}

print(len(vocab))

1132


In [ ]:
# Comprabnaod las nuevos tokens agregados
for i, item in enumerate(list(vocab.items())[-5:]):
  print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [ ]:
# Tokenizador para manejar palabras desconocidas

class SimpleTokenizerV2:
  def __init__(self, vocab):
    self.str_to_int = vocab
    self.int_to_str = {i:s for s,i in vocab.items()}

  def encode(self,text):
    preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',text)
    preprocessed =[item.strip() for item in preprocessed if item.strip()]
    # Reemplaza las palabras desconocidas por tokens <|unk|>
    preprocessed = [item if item in self.str_to_int else "<|unk>" for item in preprocessed]

    #ids = [self.str_to_int[s] for s in preprocessed], Produce error
    ids = [self.str_to_int.get(s, self.str_to_int["<|unk|>"]) for s in preprocessed]
    return ids

  def decode(self,ids):
    text = " ".join([self.int_to_str[i] for i in ids])
    text = re.sub(r'\s+([,.:;?!"()\]])', r'\1', text) # Reemplaza los espacios antes de los signos de puntuación especificados.
    return text


In [ ]:
text_1 = "Hello, do you like tea?"
text_2 = "In the sunlit terraces of the palace"
text = " <|endoftext|> ".join((text_1, text_2))
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace


In [ ]:
# Probando el nuevo tokenizador
tokenizador = SimpleTokenizerV2(vocab)
print('---------------------------------')
print(tokenizador.encode(text))


---------------------------------
[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131]


In [ ]:
print('---------------------------------')
print(tokenizador.decode(tokenizador.encode(text)))

---------------------------------
<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>


# 2.5 Byte per encoding BPE


In [ ]:
!pip install tiktoken

In [ ]:
from importlib.metadata import version
print("torch version:", version("torch"))
print("tiktoken version: ", version("tiktoken"))

torch version: 2.8.0+cu126
tiktoken version:  0.12.0


In [ ]:
# Se instancia BPE tokenizer desde tiktoken
import tiktoken
tokenizador = tiktoken.get_encoding("gpt2")

In [ ]:
# Convertri texto a IDs con BPE
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)
integers = tokenizador.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [ ]:
# Convertir IDs en texto con BPE
strings = tokenizador.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


In [ ]:
# Ejercicio 2.1
texto = "Akwirw ier"
integer=tokenizador.encode(texto)
print(integer)

[33901, 86, 343, 86, 220, 959]


In [ ]:
# Ejercicio 2.1
cadena=tokenizador.decode(integer)
print(cadena)

Akwirw ier


# 2.6 Data sampling with a sliding window

In [ ]:
with  open(file_path, 'r', encoding='utf-8') as f:
  raw_text = f.read()

enc_text = tokenizador.encode(raw_text)
print(len(enc_text))

5145


In [ ]:
enc_sample = enc_text[50:]
print(enc_sample)

[290, 4920, 2241, 287, 257, 4489, 64, 319, 262, 34686, 41976, 13, 357, 10915, 314, 2138, 1807, 340, 561, 423, 587, 10598, 393, 28537, 2014, 198, 198, 1, 464, 6001, 286, 465, 13476, 1, 438, 5562, 373, 644, 262, 1466, 1444, 340, 13, 314, 460, 3285, 9074, 13, 46606, 536, 5469, 438, 14363, 938, 4842, 1650, 353, 438, 2934, 489, 3255, 465, 48422, 540, 450, 67, 3299, 13, 366, 5189, 1781, 340, 338, 1016, 284, 3758, 262, 1988, 286, 616, 4286, 705, 1014, 510, 26, 475, 314, 836, 470, 892, 286, 326, 11, 1770, 13, 8759, 2763, 438, 1169, 2994, 284, 943, 17034, 318, 477, 314, 892, 286, 526, 383, 1573, 11, 319, 9074, 13, 536, 5469, 338, 11914, 11, 33096, 663, 4808, 3808, 62, 355, 996, 484, 547, 12548, 287, 281, 13079, 410, 12523, 286, 22353, 13, 843, 340, 373, 407, 691, 262, 9074, 13, 536, 48819, 508, 25722, 276, 13, 11161, 407, 262, 40123, 18113, 544, 9325, 701, 11, 379, 262, 938, 402, 1617, 261, 12917, 905, 11, 5025, 502, 878, 402, 271, 10899, 338, 366, 31640, 12, 67, 20811, 1, 284, 910, 11, 351, 10

In [ ]:
# Creando variables de pares entrada–objetivo
context_size = 4
X = enc_sample[:context_size]
Y = enc_sample[1:context_size+1]
print(f'X: {X}')
print(f'Y: {Y}')

X: [290, 4920, 2241, 287]
Y: [4920, 2241, 287, 257]


In [ ]:
# procesando las entradas a largo de los objetivos
for i in range(1, context_size+1):
  context = enc_sample[:i]
  disered = enc_sample[i]
  print(f' X context:{context},--------> Y desired: {disered}')

 X context:[290],--------> Y desired: 4920
 X context:[290, 4920],--------> Y desired: 2241
 X context:[290, 4920, 2241],--------> Y desired: 287
 X context:[290, 4920, 2241, 287],--------> Y desired: 257


In [ ]:
#
for i in range(1, context_size+1):
  context = enc_sample[:i]
  disered = enc_sample[i]
  print(f' X context:{tokenizador.decode(context)},--------> Y desired: {tokenizador.decode([disered])}')

 X context: and,--------> Y desired:  established
 X context: and established,--------> Y desired:  himself
 X context: and established himself,--------> Y desired:  in
 X context: and established himself in,--------> Y desired:  a


In [ ]:
# Un conjunto de datos para entradas y objetivos en lotes
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
  def __init__(self, txt, tokenizer, max_length, stride):
    self.input_ids = []
    self.target_ids = []

    tokens_ids = tokenizer.encode(txt)   # Tokeniza enteros a texto
    for i in range(0, len(tokens_ids)-max_length, stride):  # Utiliza una ventana deslizante para dividir el libro en secuencias superpuestas de longitud máxima
      input_chunk = tokens_ids[i:i+max_length]
      target_chunk = tokens_ids[i+1:i+max_length+1]
      self.input_ids.append(torch.tensor(input_chunk))
      self.target_ids.append(torch.tensor(target_chunk))

  def __len__(self):
    return len(self.input_ids)    # Regresa el total el numero de filas en el dataset

  def __getitem__(self, idx):
    return self.input_ids[idx], self.target_ids[idx]  # Regresa una fila del dataset




In [ ]:
# A data loader to generate batches with input-with pairs
def create_data_loader_v1(txt, batch_size=4,max_length=256, stride=128,
                         shuffle=True, drop_last=True, num_workers=0):
  tokenizer = tiktoken.get_encoding("gpt2")
  dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
  dataloader= DataLoader(dataset,
                         batch_size=batch_size,
                         shuffle=shuffle,
                         drop_last=drop_last,     # descarta el último lote si es más pequeño que el lote especificado,evita picos de pérdida durante el entrenamiento.
                         num_workers=num_workers)     # El número de procesos de CPU que se utilizarán
  return dataloader


In [ ]:
with open(file_path, 'r', encoding='utf-8') as f:
  raw_text = f.read()

dataloader = create_data_loader_v1(raw_text, batch_size=4, stride=1,shuffle=False)
data_iter=iter(dataloader)
first_batch = next(data_iter)
print(first_batch)


[tensor([[   40,   367,  2885,  ...,   198,  5779, 28112],
        [  367,  2885,  1464,  ...,  5779, 28112, 10197],
        [ 2885,  1464,  1807,  ..., 28112, 10197,   832],
        [ 1464,  1807,  3619,  ..., 10197,   832,   262]]), tensor([[  367,  2885,  1464,  ...,  5779, 28112, 10197],
        [ 2885,  1464,  1807,  ..., 28112, 10197,   832],
        [ 1464,  1807,  3619,  ..., 10197,   832,   262],
        [ 1807,  3619,   402,  ...,   832,   262, 46475]])]


In [ ]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 1807,  3619,   402,  ...,   832,   262, 46475],
        [ 3619,   402,   271,  ...,   262, 46475,   286],
        [  402,   271, 10899,  ..., 46475,   286, 18113],
        [  271, 10899,  2138,  ...,   286, 18113,   544]]), tensor([[ 3619,   402,   271,  ...,   262, 46475,   286],
        [  402,   271, 10899,  ..., 46475,   286, 18113],
        [  271, 10899,  2138,  ...,   286, 18113,   544],
        [10899,  2138,   257,  ..., 18113,   544,   338]])]


In [ ]:
# ejercicio 2.2-1
dataloader = create_data_loader_v1(raw_text, max_length=2, batch_size=4, stride=2,shuffle=False)
data_iter=iter(dataloader)
first_batch_exer_1 = next(data_iter)
print(first_batch_exer_1)

[tensor([[  40,  367],
        [2885, 1464],
        [1807, 3619],
        [ 402,  271]]), tensor([[  367,  2885],
        [ 1464,  1807],
        [ 3619,   402],
        [  271, 10899]])]


In [ ]:
# ejercicio 2.2-2
dataloader = create_data_loader_v1(raw_text, max_length=8, batch_size=4, stride=2,shuffle=False)
data_iter=iter(dataloader)
first_batch_exer_2 = next(data_iter)
print(first_batch_exer_2)

[tensor([[   40,   367,  2885,  1464,  1807,  3619,   402,   271],
        [ 2885,  1464,  1807,  3619,   402,   271, 10899,  2138],
        [ 1807,  3619,   402,   271, 10899,  2138,   257,  7026],
        [  402,   271, 10899,  2138,   257,  7026, 15632,   438]]), tensor([[  367,  2885,  1464,  1807,  3619,   402,   271, 10899],
        [ 1464,  1807,  3619,   402,   271, 10899,  2138,   257],
        [ 3619,   402,   271, 10899,  2138,   257,  7026, 15632],
        [  271, 10899,  2138,   257,  7026, 15632,   438,  2016]])]


In [ ]:
dataloader = create_data_loader_v1(
    raw_text, batch_size=8, max_length=4, stride=4,
    shuffle=False
)

data_iter=iter(dataloader)
inputs, targets=next(data_iter)
print("Inputs: \n ", inputs)
print("\n Targets: \n ", targets)

Inputs: 
  tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

 Targets: 
  tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


# 2.7 Creando token de embeddings

In [ ]:
inputs_id=torch.tensor([2,3,5,1])
vocab_size=6
output_dim=3

torch.manual_seed(123)
embedding_layer=torch.nn.Embedding(vocab_size,output_dim)
print(embedding_layer)
print('---------------------')
print(embedding_layer.weight)
print('---------------------')

Embedding(6, 3)
---------------------
Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)
---------------------


In [ ]:
# Obtner el vector de embedding
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


# 2.8 Codificar palabras de position

In [ ]:
#token_embedding
vocab__size=50257
output_dim= 256

token_embedding_layer= torch.nn.Embedding(vocab__size, output_dim)

In [ ]:
max_length= 4
dataloader= create_data_loader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
    )
data_iter= iter(dataloader)
inputs,targets=next(data_iter)
print("Token IDs: \n", inputs)
print('-------------------------------------------------------')
print('\nInputs shape: \n', inputs.shape)
print('-------------------------------------------------------')

Token IDs: 
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
-------------------------------------------------------

Inputs shape: 
 torch.Size([8, 4])
-------------------------------------------------------


In [ ]:
# uso de la capa de embedding,
token_embeddings=token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [ ]:
#embedding posicional absoluto de un modelo GPT,
#solo necesitamos crear otra capa de
#embedding que tenga la misma dimensión que la capa token_embedding
#
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [ ]:
#añadirá el tensor pos_embeddings de dimensión 4 × 256 a cada
#tensor de embedding de tokens de dimensión 4 × 256
#en cada uno de los ocho lotes.

input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)


torch.Size([8, 4, 256])


# Bonus material

#### Comparing Various Byte Pair Encoding (BPE) Implementations

In [ ]:
from importlib.metadata import version

print("tiktoken version:", version("tiktoken"))

tiktoken version: 0.12.0


In [ ]:
import tiktoken

tik_tokenizer = tiktoken.get_encoding("gpt2")

text = "Hello, world. Is this-- a test?"

In [ ]:
integers = tik_tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 995, 13, 1148, 428, 438, 257, 1332, 30]


In [ ]:
strings = tik_tokenizer.decode(integers)
print(strings)

Hello, world. Is this-- a test?


In [ ]:
print(tik_tokenizer.n_vocab)

50257


### Comprendiendo la diferencia entre capas de embedding y capas lineales

Las capas de embedding en PyTorch logran lo mismo que las capas lineales que realizan multiplicaciones matriciales; la razón por la que usamos capas de embedding es por eficiencia computacional.
Vamos a analizar esta relación paso a paso utilizando ejemplos de código en PyTorch.


In [ ]:
import torch
print("PyTorch version:", torch.__version__)

PyTorch version: 2.8.0+cu126


In [ ]:
# Supongamos que tenemos los siguientes 3 ejemplos de entrenamiento,
# que pueden representar IDs de tokens en el contexto de un LLM
idx = torch.tensor([2, 3, 1])

# El número de filas en la matriz de embedding se puede determinar
# obteniendo el ID de token más grande + 1.
# Si el ID de token más alto es 3, entonces queremos 4 filas, para los posibles
# IDs de token 0, 1, 2, 3
num_idx = max(idx)+1

# La dimensión deseada del embedding es un hiperparámetro
out_dim = 5


In [ ]:
# Usamos una semilla aleatoria para garantizar la reproducibilidad, ya que
# los pesos en la capa de embedding se inicializan con
# pequeños valores aleatorios
torch.manual_seed(123)

embedding = torch.nn.Embedding(num_idx, out_dim)


In [ ]:
embedding.weight

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.3035, -0.5880,  1.5810],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015],
        [ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953]], requires_grad=True)

In [ ]:
#Podemos entonces usar las capas de embedding para obtener la representación vectorial de un ejemplo
#de entrenamiento con ID 1:
embedding(torch.tensor([1]))


tensor([[ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]],
       grad_fn=<EmbeddingBackward0>)

In [ ]:
# Similarmente con el ID 2
embedding(torch.tensor([2]))

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315]],
       grad_fn=<EmbeddingBackward0>)